In [37]:
# kernel hamne pranay wala anaconda setup kiya hai for future reference
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
import json
import pickle

In [39]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD

In [40]:
import random

words=[]
classes=[]
documents=[]
ignore_words=['?','!']
data_file = open(r'D:\target\ml\ChatBot\intents.json').read()
intents = json.loads(data_file)


In [57]:

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern, preserve_line=True)

        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

1820 documents
255 classes ['     Support Vector Machine', '   Decision Trees', '  Perceptron', '  Regularization in Machine Learning', '  applications of supervised learning', '  basic data structures', " 'action' in reinforcement learning", " 'agent' in reinforcement learning", " 'value' in reinforcement learning", ' Classification model', ' Gradient Descent', ' Inductive Logic Programming', ' Polynomial Regression', ' Regularization', ' Ridge Regression technique', ' Types of machine learning', ' advantages of using Polynomial Regression over Linear Regression', ' applications of self-supervised learning', ' components of Bayesian logic program', ' difference between heuristic for rule learning and heuristics', ' disadvantages of supervised learning', ' imbalanced datasheet', ' issues to consider in supervised learning', ' standard supervised learning', 'A/B Testing', 'Algorithm techniques in Machine Learning', 'Appreciation', 'Apriori algorithm', 'Areas of Problems', 'BiasML', 'Bin

In [60]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))



In [62]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # Initialize bag of words
    bag = []
    # Tokenized pattern words
    pattern_words = doc[0]
    # Lemmatize and lower-case each word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Create bag of words
    bag = [1 if w in pattern_words else 0 for w in words]
    
    # Ensure bag matches words length
    if len(bag) != len(words):
        print(f"Bag length mismatch: expected {len(words)}, got {len(bag)}")
        continue  # Skip problematic entry

    # Output row initialization
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Ensure output row matches classes length
    if len(output_row) != len(classes):
        print(f"Output row length mismatch: expected {len(classes)}, got {len(output_row)}")
        continue  # Skip problematic entry

    training.append([bag, output_row])

# Shuffle training data
random.shuffle(training)

# Ensure consistent data structure before conversion
training = np.array(training, dtype=object)

# Split into train_x and train_y
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])


In [69]:
import tensorflow as tf
print(tf.__version__)  # Ensure you're using the correct version

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=400, batch_size=6, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

2.18.0
Epoch 1/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0087 - loss: 5.5405
Epoch 2/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0165 - loss: 5.4236
Epoch 3/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0354 - loss: 5.2059
Epoch 4/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0821 - loss: 4.7428
Epoch 5/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1148 - loss: 4.2314
Epoch 6/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1764 - loss: 3.7894
Epoch 7/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2261 - loss: 3.3196
Epoch 8/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2793 - loss: 2.9776
Epoch 9/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3108 - loss: 2.7698
Epoch 10/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3360 - loss: 2.5666
Epoch 11/400
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3775 - loss: 2.3513
Epoch 12/400
304/304 ━━━━━━━━━━━━━

model created
